In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install ultralytics
%pip install scikit-learn

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import numpy as np
import cv2
import shutil
import yaml
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 42
BATCH_SIZE = 8
MODEL = "Yolov8"
data_path = '/...'

In [ ]:
#model = YOLO(f"{data_path}/Yolov8_v3/v3/train/weights/last.pt")
model = YOLO("yolov8x") # yolov8n, yolov8s, yolov8m, yolov8l, yolov8x 등이 있음
results = model.train(
    data= data_path + "/data/yolocustom.yaml",
    imgsz=640,
    epochs=50,
    batch=BATCH_SIZE,   # 한 번의 훈련 단계에서 사용할 배치 크기
    patience=5,         # 성능 향상이 없을 때, 5 에포크 동안 성능이 향상되지 않으면 훈련이 조기 종료
    workers=16,         # 데이터 로딩을 위한 워커 수를 지정
    device=0,           # 훈련에 사용할 GPU 장치를 지정. 여기서는 첫 번째 GPU(즉, device=0)를 사용
    exist_ok=True,      # 지정된 프로젝트와 이름을 사용하는 디렉토리가 이미 존재해도 덮어쓰기를 허용
    project=f"{data_path}/{MODEL}",
    name="train",
    seed=SEED,
    pretrained=False,   # 사전 훈련된 가중치를 사용하지 않음을 지정
    resume=False,       # 이전 훈련을 이어서 하지 않음을 지정

    optimizer="AdamW",
    lr0=0.0005, 
    mixup=0.3,          # 믹스업(data augmentation 기법)의 비율을 0.3으로 설정. 이는 두 이미지와 레이블을 섞는 비율을 의미
    cos_lr=True,        # 코사인 학습률 스케줄러를 사용하여 학습률을 조정
    flipud=0.0,         # 이미지 상하 반전 확률을 0.0으로 설정
    fliplr=0.0,         # 이미지 좌우 반전 확률을 0.0으로 설정

    augment=False,      # 데이터 증강을 사용하지 않음
    val=True,           # 훈련 중 검증을 수행함
    cache=True          # 데이터셋을 캐시하여 로딩 속도를 향상
    )

In [ ]:
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO(f"{data_path}/{MODEL}/train/weights/best.pt")
test_image_paths = glob(data_path + "/data/yolo_v8/test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(640, 640), iou=0.8, conf=0.3, save_conf=True, save=False, save_txt=True, project=f"{data_path}/{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
            # iou: IoU (Intersection over Union) 임계값. 0~1 사이의 값을 가짐. 정답 바운딩 박스와 예측한 바운딩 박스의 교집합 비율을 나타냄.
            #      두 바운딩 박스의 교집합 / 두 바운딩 박스의 합집합
            # conf: 객체로 간주할 확률(confidence)의 임계값. 이 값보다 높은 확률을 가진 바운딩 박스들만 최종 결과에 포함됨
            # save_conf: 예측 결과에서 각 객체의 확률(confidence) 값을 저장할지 여부
            # save: 예측 결과 이미지 저장 여부
            # save_txt: 예측 결과를 텍스트 파일로 저장할지 여부
    if i % 1 == 0:
        clear_output(wait=True)

In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = (x - width / 2) * image_width
    x_max = (x + width / 2) * image_width
    y_min = (y - height / 2) * image_height
    y_max = (y + height / 2) * image_height

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
infer_txts = glob(f"{data_path}/{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"{data_path}/data/yolo_v8/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"{data_path}/results/{MODEL}.csv", index=False)